# Crane V1 training evolution visualisation

#### Imports

In [7]:
import gym
import numpy as np

env = gym.make('crane-v0') #Load the environement

#### DQN Agent

In [8]:
import numpy as np
import random
from collections import namedtuple, deque

#from model import QNetwork # UNCOMMENT IF YOU ARE NOT IN A JUPYTER NOTEBOOK

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = 5*int(1e5)   # replay buffer size
BATCH_SIZE = 64            # minibatch size
GAMMA = 0.995 #was 0.99    # discount factor
TAU = 1e-3                 # for soft update of target parameters
LR =  5e-4                 # learning rate 
UPDATE_EVERY = 4           # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """

        
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)       

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

#### Neural Network

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64): 
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        


    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

#### Initialize agent

In [10]:
import torch
import time
from collections import deque
#from agent import Agent   # UNCOMMENT IF YOU ARE NOT IN A JUPYTER NOTEBOOK
import matplotlib.pyplot as plt
%matplotlib inline

state_size=4        #State size of  environement 
action_size=3       #Action size of enviroenemnt
seed=0
agent = Agent(state_size=4, action_size=3, seed=0) #setting the agent's parameters 

### Training Visualization

#### No training, random actions

In [13]:
#Choose model
#agent.qnetwork_local.state_dict() 

observation = env.reset()
reward_sum = 0
for t in range(200):
    env.render()
    time.sleep(0.01)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)

    reward_sum = reward_sum + reward
        
    if done:
        break
env.close()


#### 100th episode of training

Takes random actions but aims for x-target

In [15]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_100.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final angle velocity is : ", state[3])
env.close()


Score: -3030.569890800945
final state is :  [ 32.38880653   1.24270697 100.49079694   4.8826199 ]
final x position is :  32.388806529398444
final angle velocity is :  4.8826198975026305


#### 200th episode of training

Aims for target and try to stabilyse but still takes random actions and diverges

In [17]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_200.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final angle velocity is : ", state[3])
env.close()


Score: 339.6944131441556
final state is :  [0.28787257 1.35571766 2.79308925 0.43629742]
final x position is :  0.2878725740546435
final angle velocity is :  0.43629742099058927


#### 400th episode of training

Understands that no swing arround target means more rewards

In [19]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_400.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final angle velocity is : ", state[3])
env.close()


Score: 7659.958621809027
final state is :  [ 1.02799059  0.36521425  3.0367613  -0.42804797]
final x position is :  1.0279905921517785
final angle velocity is :  -0.42804797056392263


#### 500th episode of training

Learns that going slow can prevent swaying

In [21]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_500.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final angle velocity is : ", state[3])
env.close()


Score: 1965.7378319112154
final state is :  [ 1.09233836 -0.37618105  3.2259113   0.09380719]
final x position is :  1.0923383604723935
final angle velocity is :  0.09380719207502294


#### 800th episode of training

The agent starts to find the solution and performs better and faster at going to it by going fast to the x-target and then stabilysing

In [23]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_800.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final angle velocity is : ", state[3])
env.close()


Score: 11244.505699620322
final state is :  [1.1285223  0.05441157 3.24144299 0.25096817]
final x position is :  1.1285222950578537
final angle velocity is :  0.2509681677579218


#### Solved training

Solved

In [28]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('checkpoint_precise.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final angle velocity is : ", state[3])
env.close()


Total steps :  394.0
Total time is :  7.88
Score: 105329.15706783575
final state is :  [ 1.0082154  -0.01223609  3.16362433  0.02906195]
final x position is :  1.008215401707017
final angle velocity is :  0.029061947237530278
